In [1]:
from simply import redshift
from scipy import stats
import numpy as np

In [58]:
social_proof_test = redshift("""
WITH social_proof_lp_test as (
SELECT DISTINCT
	CASE WHEN variation_id = 14490420253 THEN 'original' 
			 WHEN variation_id = 14494190727 THEN 'social proof' 
			 ELSE NULL 
			 END test_group,
	du.user_ref,
	first_account_open_date
FROM fact_client_event fce
JOIN (SELECT user_ref, MIN(event_ts) AS first_event_optimizely
		  FROM fact_client_event
			WHERE experiment_id = 14482520534
			GROUP BY 1) feo ON fce.user_ref = feo.user_ref AND fce.event_ts = feo.first_event_optimizely
JOIN dim_user du on fce.user_ref = du.user_ref AND fce.event_ts < du.first_account_open_ts
WHERE experiment_id = 14482520534
    AND first_channel = 'PAID'
	)

SELECT
	lp.user_ref,
	test_group,
	first_account_open_date,
	balance_eod,
	running_total_swipe_count,
    CASE WHEN running_total_swipe_count >= 1 THEN 1 ELSE 0 END AS swipe_flag,
	ft.amount AS first_deposit_amount,
	CASE WHEN first_deposit_ts::date IS NOT NULL THEN 1 ELSE 0 END AS deposit_flag,
    CASE WHEN has_individual_protected_account = true THEN 1 ELSE 0 END protected_flag,
    individual_protected_balance_eod
FROM social_proof_lp_test lp
JOIN fact_user_day fud on lp.user_ref = fud.user_ref
LEFT JOIN (SELECT user_ref, MIN(transaction_ts) AS first_deposit_ts
					 FROM fact_transaction ft
					 WHERE is_deposit = true
							AND amount > 1
					 GROUP BY 1) d ON fud.user_ref = d.user_ref AND first_deposit_ts::date <= first_account_open_date + 14
LEFT JOIN fact_transaction ft ON d.user_ref = ft.user_ref AND d.first_deposit_ts = ft.transaction_ts
WHERE num_open_account_days = 14
    AND running_total_swipe_count < 50;
""")

In [59]:
social_proof_test.describe()

,balance_eod,running_total_swipe_count,swipe_flag,first_deposit_amount,deposit_flag,protected_flag,individual_protected_balance_eod
count,2465.000000,2465.000000,2465.000000,481.000000,2465.000000,2465.000000,2465.000000
mean,105.360450,0.468560,0.081947,470.504990,0.195132,0.065314,43.430454
std,626.529497,2.341685,0.274340,1007.531912,0.396383,0.247130,387.717208
min,-40.520000,0.000000,0.000000,1.310000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,15.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,54.730000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,334.590000,0.000000,0.000000,0.000000
max,11640.530000,37.000000,1.000000,5000.000000,1.000000,1.000000,7085.660000


In [60]:
social_proof = social_proof_test[social_proof_test.test_group == 'social proof']
control = social_proof_test[social_proof_test.test_group == 'original']

In [61]:
social_proof.describe()

,balance_eod,running_total_swipe_count,swipe_flag,first_deposit_amount,deposit_flag,protected_flag,individual_protected_balance_eod
count,1203.000000,1203.000000,1203.000000,238.000000,1203.000000,1203.000000,1203.000000
mean,103.994755,0.507897,0.080632,429.095462,0.197839,0.058188,34.197714
std,663.204145,2.488940,0.272382,889.087671,0.398536,0.234196,335.875943
min,-40.520000,0.000000,0.000000,1.310000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,16.997500,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,57.365000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,400.000000,0.000000,0.000000,0.000000
max,11640.530000,28.000000,1.000000,5000.000000,1.000000,1.000000,5000.000000


In [62]:
control.describe()

,balance_eod,running_total_swipe_count,swipe_flag,first_deposit_amount,deposit_flag,protected_flag,individual_protected_balance_eod
count,1262.000000,1262.000000,1262.000000,243.000000,1262.000000,1262.000000,1262.000000
mean,106.662298,0.431062,0.083201,511.062469,0.192552,0.072108,52.231553
std,589.711463,2.192465,0.276296,1111.746115,0.394460,0.258769,431.325706
min,-30.010000,0.000000,0.000000,1.410000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,12.600000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,300.000000,0.000000,0.000000,0.000000
max,7090.660000,37.000000,1.000000,5000.000000,1.000000,1.000000,7085.660000


## T Test for Balance

In [63]:
tt = stats.ttest_ind(social_proof.balance_eod, control.balance_eod, nan_policy='omit')
print("Balance: T-Test for equal means: p-value = {}".format(tt.pvalue))

Balance: T-Test for equal means: p-value = 0.9158749667690324


## Chi Square Deposit Flag

In [64]:
# Calculation Unique Contact - Chi Square Test
obs = np.array([[social_proof[social_proof.deposit_flag ==1].user_ref.count(),
                 social_proof[social_proof.deposit_flag ==0].user_ref.count()],
                [control[control.deposit_flag ==1].user_ref.count(),
                 control[control.deposit_flag ==0].user_ref.count()]])

chi2, p, dof, expected = stats.chi2_contingency(obs)

print("Deposit Flag Day 14 - Chi2 Test for Proportions: p-value = {}".format(p))

Deposit Flag Day 14 - Chi2 Test for Proportions: p-value = 0.7792785399166784


## T Test for First Deposit Amount

In [65]:
tt = stats.ttest_ind(social_proof.first_deposit_amount, control.first_deposit_amount, nan_policy='omit')
print("Balance: T-Test for equal means: p-value = {}".format(tt.pvalue))

Balance: T-Test for equal means: p-value = 0.37290465895744973


## Chi Square Swipe Flag

In [67]:
# Calculation Unique Contact - Chi Square Test

obs = np.array([[social_proof[social_proof.swipe_flag == 1].user_ref.count(),
                 social_proof[social_proof.swipe_flag ==0].user_ref.count()],
                [control[control.swipe_flag == 1].user_ref.count(),
                 control[control.swipe_flag ==0].user_ref.count()]])

chi2, p, dof, expected = stats.chi2_contingency(obs)

print("Deposit Flag Day 14 - Chi2 Test for Proportions: p-value = {}".format(p))

Deposit Flag Day 14 - Chi2 Test for Proportions: p-value = 0.8736405766165148


## T Test for Number of Swipes

In [68]:
tt = stats.ttest_ind(social_proof.running_total_swipe_count, control.running_total_swipe_count, nan_policy='omit')
print("Swipes Day 14: T-Test for equal means: p-value = {}".format(tt.pvalue))

Swipes Day 14: T-Test for equal means: p-value = 0.41558272820664466


## Chi Square Protected Goal Flag

In [69]:
# Calculation Unique Contact - Chi Square Test

obs = np.array([[social_proof[social_proof.protected_flag == 1].user_ref.count(),
                 social_proof[social_proof.protected_flag ==0].user_ref.count()],
                [control[control.protected_flag == 1].user_ref.count(),
                 control[control.protected_flag ==0].user_ref.count()]])

chi2, p, dof, expected = stats.chi2_contingency(obs)

print("Protected Goal Flag - Chi2 Test for Proportions: p-value = {}".format(p))

Protected Goal Flag - Chi2 Test for Proportions: p-value = 0.187970773787099


## T Test for Protected Goal Balance

In [40]:
tt = stats.ttest_ind(social_proof.individual_protected_balance_eod, control.individual_protected_balance_eod, nan_policy='omit')
print("Individual Protected Balance: T-Test for equal means: p-value = {}".format(tt.pvalue))

Individual Protected Balance: T-Test for equal means: p-value = 0.24844806026579042
